In [1]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
import random
import joblib
import sympy
import math
from math import e
import random
from keras import Model
from keras.models import load_model
import numpy as np
import os
import pandas as pd

2021-11-24 20:43:37.293187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-24 20:43:37.293223: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed'
    }
TSS_HK = []

with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3']:
            TSS_HK.append(TSS(ll[0], int(int(ll[1])+1000)))

TSS_NonGene = []
with open(dic['non_gene'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        TSS_NonGene.append(TSS(ll[0], int(ll[1])+int(1000)))

bamfile = ps.AlignmentFile(dic['bam123'],'rb')

TSS_NonGene = TSS_NonGene[:len(TSS_HK)]

TSSes_x = TSS_HK + TSS_NonGene
TSSes_y = [1]*len(TSS_HK) + [0]*len(TSS_NonGene)
perm = random.sample(range(len(TSSes_x)),len(TSSes_x))
TSSes_x = np.array(TSSes_x)
TSSes_y = np.array(TSSes_y)
TSSes_x = TSSes_x[perm[:len(perm)]]
TSSes_y = TSSes_y[perm[:len(perm)]]
labels = TSSes_y
up = 1000
down = 1000

In [3]:
#lstm
win = 40
feature_matrix = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    wps = np.zeros(up+down, dtype=float)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
            # wps_total
            region1 = int(max(0, ss + win/2))
            region2 = int(min(ee - win/2, end-start))
            i = region1
            while i < region2:
                wps[i] += 1
                i = i+1
            # wps_part
            region1 = int(max(0, ss - win/2))
            region2 = int(min(end-start, ss + win/2))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i + 1
            # wps_part
            region1 = int(max(ee - win/2, 0))
            region2 = int(min(ee + win/2, end-start))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i+1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        ff.append(int(round(np.mean(wps[ss:ee]))))
        feature_win[k] = np.array(ff)
        k = k + 1
    feature_matrix.append(feature_win)
feature_matrix = np.array(feature_matrix)

lstm_x = []
for mat in feature_matrix:
    lstm_x.append(mat)
lstm_x = np.array(lstm_x)
train_x = lstm_x[:int(len(lstm_x)*0.8)]
train_y = labels[:int(len(labels)*0.8)]
test_x = lstm_x[int(len(lstm_x)*0.8):]
test_y = labels[int(len(labels)*0.8):]



In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
input_size = 4
time_steps = 40
cell_size = 20

model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True),input_shape=(lstm_x.shape[1], lstm_x.shape[2])))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None))
model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(16, activation='relu', kernel_regularizer=None, bias_regularizer=None))
# model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None))

# print(model.summary())

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(lstm_x, labels, epochs=300, batch_size=128, validation_split=0.1,
                        callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./' + 'lstm_4_feature.h5', save_best_only=True)],
                        shuffle=True, verbose=2)
# from sklearn import metrics
# pred_lstm = model.predict(test_x)
# fpr, tpr, thresholds = metrics.roc_curve(test_y, pred_lstm, pos_label=1)
# auc = metrics.auc(fpr, tpr)
# plt.figure()
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.legend(loc='lower right')
# plt.show()
# loss, accuracy = model.evaluate(test_x, test_y)
# print("loss=",loss, "acc=",accuracy)

Epoch 1/300
12/12 - 3s - loss: 0.4967 - accuracy: 0.7748 - val_loss: 0.3155 - val_accuracy: 0.8902
Epoch 2/300
12/12 - 1s - loss: 0.2693 - accuracy: 0.9152 - val_loss: 0.2349 - val_accuracy: 0.9207
Epoch 3/300
12/12 - 1s - loss: 0.2256 - accuracy: 0.9294 - val_loss: 0.2140 - val_accuracy: 0.9207
Epoch 4/300
12/12 - 1s - loss: 0.2059 - accuracy: 0.9294 - val_loss: 0.2014 - val_accuracy: 0.9085
Epoch 5/300
12/12 - 1s - loss: 0.2000 - accuracy: 0.9328 - val_loss: 0.1991 - val_accuracy: 0.9207
Epoch 6/300
12/12 - 1s - loss: 0.1905 - accuracy: 0.9423 - val_loss: 0.1944 - val_accuracy: 0.9207
Epoch 7/300
12/12 - 1s - loss: 0.1802 - accuracy: 0.9444 - val_loss: 0.1905 - val_accuracy: 0.9268
Epoch 8/300
12/12 - 1s - loss: 0.1747 - accuracy: 0.9478 - val_loss: 0.1921 - val_accuracy: 0.9146
Epoch 9/300
12/12 - 1s - loss: 0.1713 - accuracy: 0.9450 - val_loss: 0.1947 - val_accuracy: 0.9268
Epoch 10/300
12/12 - 1s - loss: 0.1689 - accuracy: 0.9450 - val_loss: 0.1874 - val_accuracy: 0.9207
Epoch 11/

[21, 45, 58, 59, 84, 85, 90, 130, 153, 165, 166, 184, 189, 204, 223, 235, 238, 264, 269, 272, 304, 308]


In [4]:
'''cnn_input'''
raw_data = []
win = 120
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    wps = np.zeros(up+down, dtype=float)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
            # wps_total
            region1 = int(max(0, ss + win/2))
            region2 = int(min(ee - win/2, end-start))
            i = region1
            while i < region2:
                wps[i] += 1
                i = i+1
            # wps_part
            region1 = int(max(0, ss - win/2))
            region2 = int(min(end-start, ss + win/2))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i + 1
            # wps_part
            region1 = int(max(ee - win/2, 0))
            region2 = int(min(ee + win/2, end-start))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i+1
    raw_data.append([cov,long-short,up_end-down_end,wps])
raw_data = np.array(raw_data)

cnn_x = []
for mat in raw_data:
    cnn_x.append(mat)
cnn_x = np.array(cnn_x)
cnn_x = cnn_x.reshape(cnn_x.shape[0], cnn_x.shape[1], cnn_x.shape[2], 1)
train_x2 = cnn_x[:int(len(cnn_x)*0.8)]
train_y2 = labels[:int(len(labels)*0.8)]
test_x2 = cnn_x[int(len(cnn_x)*0.8):]
test_y2 = labels[int(len(labels)*0.8):]
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

LL = keras.utils.to_categorical(test_y2)
import random
model = keras.models.Sequential()

model.add(layers.Conv2D(filters=32,kernel_size=(2,2),strides=(1,1),padding='same',input_shape=(cnn_x.shape[1], cnn_x.shape[2],1),name='0'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(strides=(2,2),pool_size=(2,2),name='1'))
model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',name='2'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(strides=(2,2),pool_size=(2,2),name='3'))
model.add(layers.Flatten())
model.add(layers.Dense(300,activation='relu',kernel_regularizer=None, bias_regularizer=None,name='5'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(16,activation='relu',kernel_regularizer=None, bias_regularizer=None,name='6'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=None, bias_regularizer=None,name='7'))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(train_x2, train_y2, epochs=300, batch_size=128, validation_split=0.1,
                        callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./' + 'weight.h5', save_best_only=True)],
                        shuffle=True, verbose=2)
from sklearn import metrics
pred_cnn = model.predict(test_x2)

fpr, tpr, thresholds = metrics.roc_curve(test_y2, pred_cnn, pos_label=1)
auc = metrics.auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
loss, accuracy = model.evaluate(test_x2, test_y2)
print("loss=",loss, "acc=",accuracy)

KeyboardInterrupt: 